In [1]:
from datasets import load_dataset
from .autonotebook import tqdm as notebook_tqdm

/Users/apple/Desktop/KamalRaj/Assignments/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("glue", "mrpc", split="train") # load a config from glue dataset - only train split

Generating test split: 100%|██████████| 1725/1725 [00:00<00:00, 146239.00 examples/s]


In [ ]:
### Only comments
